In [1]:
import pandas as pd

df_rma = pd.read_csv("data/Diag/microarray/data_rma.csv.gz", index_col=0)
df_norm = pd.read_csv("data/Diag/microarray/data_norm.csv.gz", index_col=0)
df_debat = pd.read_csv("data/Diag/microarray/data_debat.csv.gz", index_col=0)

label = pd.read_csv("data/Diag/microarray/data_labels.csv", index_col=0)

In [2]:
lb = label

train_index = lb[((lb["GSE_ID"].isin(["GSE30999", "GSE13355", "GSE03189"])) & (lb["Labels_5class"] == "Healthy") |
                  (lb["GSE_ID"].isin(["GSE07553", "GSE42109"])) & (lb["Labels_5class"] == "BCC") |
                  (lb["GSE_ID"].isin(["GSE45216", "GSE07553"])) & (lb["Labels_5class"] == "SCC") |
                  (lb["GSE_ID"].isin(["GSE39612"])) & (lb["Labels_5class"] == "MCC") |
                  (lb["GSE_ID"].isin(["GSE07553", "GSE15605", "GSE03189"])) & (lb["Labels_5class"] == "MEL"))].index
test_index = lb[~lb.index.isin(train_index)].index

In [3]:
relate_genes = pd.read_table("data/Diag/NCBI_skin_cancer_gene.txt")["Symbol"]
intersection_genes = df_rma.index.intersection(relate_genes)

train_set_nonorm = df_rma[train_index]
test_set_nonorm = df_rma[test_index]

train_set_comb = df_debat[train_index]
test_set_comb = df_debat[test_index]

train_set_norm = df_norm[train_index]
test_set_norm = df_norm[test_index]

In [6]:
from pandarallel import pandarallel
from scipy.stats import chi2_contingency
from utils import get_genepairs
from fisher import pvalue_npy
import numpy as np

for a in [0,1,2]:
    for i in ["5class", "3class","2class"]:
        pairs = get_genepairs(train_set_norm.T[intersection_genes], label.loc[train_index][f"Labels_{i}"], 200000, a)
        if i != "2class":
            pandarallel.initialize()
            def cal_P(x):
                import numpy as np
                from scipy.stats import chi2_contingency
                return chi2_contingency((np.array(
                            [x[x.index[x.index.str.contains('_1')]].values,
                             x[x.index[x.index.str.contains('_-1')]].values]) + 1).transpose(1, 0)).pvalue
            pairs["p"]=pairs.parallel_apply(cal_P,axis=1)

        else:
            c=pairs.columns.map(lambda x: x.split("_")[0])[2:].unique()
            _, _, twosided = pvalue_npy(pairs[f'{c[0]}_1'].values.astype(np.uint),
                                        pairs[f'{c[0]}_-1'].values.astype(np.uint),
                                        pairs[f'{c[1]}_1'].values.astype(np.uint),
                                        pairs[f'{c[1]}_-1'].values.astype(np.uint))
            pairs["p"] = twosided
        pairs.to_csv(f"result/Diag/norm/pairs_{i}_a{a}_norm.csv")

100%|██████████| 1137786/1137786 [01:45<00:00, 10806.34it/s]


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


100%|██████████| 1137786/1137786 [01:15<00:00, 15108.91it/s]


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


100%|██████████| 1137786/1137786 [01:33<00:00, 12161.59it/s]


...

100%|██████████| 1137786/1137786 [01:32<00:00, 12322.03it/s]


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


100%|██████████| 1137786/1137786 [00:58<00:00, 19569.90it/s]


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


100%|██████████| 1137786/1137786 [00:41<00:00, 27686.68it/s]


In [7]:
from utils import dftopair
from collections import defaultdict
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

for a1 in [0,1,2]:
    for lbs in ["Labels_2class", "Labels_5class", "Labels_3class"]:
        pairs=pd.read_csv(f"result/Diag/norm/pairs_{lbs.split('_')[1]}_a{a1}_norm.csv").sort_values(by="p")
        result_df=pd.DataFrame()
        for p in range(10,110,10):
            str2numlabel_dict = dict(zip(label[lbs].unique(), range(len(label[lbs].unique()))))
            
            x_train=dftopair(train_set_norm.T,pairs[:p]["pairs0"],pairs[:p]["pairs1"])
            x_test=dftopair(test_set_norm.T,pairs[:p]["pairs0"],pairs[:p]["pairs1"])
            y_train = lb[lbs].map(str2numlabel_dict)[train_index]
            y_test = lb[lbs].map(str2numlabel_dict)[test_index]
    
            classifers=[RandomForestClassifier(), SVC(),
                    GaussianNB(), KNeighborsClassifier(), LogisticRegression(),LGBMClassifier(verbosity= -1),XGBClassifier(),MLPClassifier()]
            
           
            result = defaultdict(dict)
            for classifier in classifers:
                classifier.fit(x_train, y_train)
                y_pred = classifier.predict(x_test)
                result[classifier.__repr__()] = {
                    "acc": accuracy_score(y_test, y_pred),
                    "precision_weighted": precision_score(y_test, y_pred, average="weighted"),
                    "precision_macro": precision_score(y_test, y_pred, average="macro"),
                    "f1_weighted": f1_score(y_test, y_pred, average="weighted"),
                    "f1_macro": f1_score(y_test, y_pred, average="macro"),
                    "recall_weighted": recall_score(y_test, y_pred, average="weighted"),
                    "recall_macro": recall_score(y_test, y_pred, average="macro")}
            result = pd.DataFrame(result)
            result["pairs_num"]=p
            result_df=pd.concat([result_df,result])
        result_df.to_csv(f"result/Diag/norm/result_{lbs.split('_')[1]}_a{a1}.csv")

F:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
F:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
F:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
F:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
F:\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar